In [1]:
from fastai.vision.all import *

In [2]:
path = untar_data(URLs.IMAGENETTE)

In [3]:
dblock = DataBlock(blocks = (ImageBlock(), CategoryBlock()),
                   get_items = get_image_files,
                   get_y = parent_label,
                   item_tfms= Resize(460),
                   batch_tfms = aug_transforms(size = 224, min_scale = 0.75))
dls = dblock.dataloaders(path, bs = 128)

dataset 축소 (ImageNette 사용)

In [4]:
model = xresnet50(n_out = dls.c)
learn = Learner(dls, model, loss_func = CrossEntropyLossFlat(), metrics = accuracy)
learn.fit_one_cycle(5, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.614102,4.088572,0.268857,02:42
1,1.336020,1.352478,0.563107,02:38
2,1.042501,1.059920,0.653846,02:40
3,0.825907,0.816535,0.743092,02:58
4,0.683131,0.610278,0.806946,02:48


In [6]:
x,y = dls.one_batch()
x.mean(dim=[0,2,3]),  x.std(dim=[0,2,3])

(TensorImage([0.4631, 0.4581, 0.4316], device='cuda:0'),
 TensorImage([0.2774, 0.2766, 0.2986], device='cuda:0'))

Normalization (이미지넷 mean, std)

In [8]:
dblock = DataBlock(blocks = (ImageBlock(), CategoryBlock()),
                   get_items = get_image_files,
                   get_y = parent_label,
                   item_tfms= Resize(460),
                   batch_tfms = [*aug_transforms(size = 224, min_scale = 0.75),
                                 Normalize.from_stats(*imagenet_stats)])
dls = dblock.dataloaders(path, bs = 128)

In [10]:
x,y = dls.one_batch()
x.mean(dim=[0,2,3]),  x.std(dim=[0,2,3])

(TensorImage([-0.0428,  0.0221,  0.0983], device='cuda:0'),
 TensorImage([1.2441, 1.2571, 1.3521], device='cuda:0'))

In [12]:
model = xresnet50()
learn = Learner(dls, model, loss_func = CrossEntropyLossFlat(), metrics = accuracy)
learn.fit_one_cycle(5,3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.882333,2.232546,0.380508,02:40
1,1.330479,1.708718,0.483196,02:39
2,1.042483,1.692178,0.573562,02:57
3,0.822342,0.698175,0.778193,02:45
4,0.659835,0.594195,0.810306,02:40


점진적 크기 조절(Progressive Resize)

In [13]:
def get_dls(bs,size):
    dblock = DataBlock(blocks = (ImageBlock(), CategoryBlock()),
                    get_items = get_image_files,
                    get_y = parent_label,
                    item_tfms= Resize(460),
                    batch_tfms = [*aug_transforms(size = size, min_scale = 0.75),
                                    Normalize.from_stats(*imagenet_stats)])
    return dblock.dataloaders(path, bs=bs)

In [14]:
dls = get_dls(128,128)
learn = Learner(dls, model, loss_func = CrossEntropyLossFlat(), metrics = accuracy)
learn.fit_one_cycle(4,3e-3)
learn.dls  = get_dls(64,224)
learn.fine_tune(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.888471,3.308702,0.402166,02:12
1,0.852185,1.998049,0.579910,02:12
2,0.699165,0.592962,0.812547,02:19
3,0.546958,0.499824,0.845034,02:12


epoch,train_loss,valid_loss,accuracy,time
0,0.598881,0.503675,0.844660,02:50


epoch,train_loss,valid_loss,accuracy,time
0,0.480919,0.423834,0.859597,02:42
1,0.492137,0.541544,0.827483,02:45
2,0.443613,0.400723,0.874907,02:44
3,0.399631,0.333676,0.896191,02:39
4,0.337961,0.320303,0.900672,02:40


Test Time Augmentation

In [21]:
# for i in range(10):
#     print(f'{i}th test')
preds, targs = learn.tta()
accuracy(preds, targs).item()

0.8252426981925964

Mix up

In [17]:
model = xresnet50(n_out = dls.c)
learn = Learner(dls, model, loss_func = CrossEntropyLossFlat(),
                metrics = accuracy, cbs = MixUp())
learn.fit_one_cycle(5,3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.872651,2.523620,0.376774,02:11
1,1.652667,1.577164,0.526512,02:12
2,1.446681,1.693870,0.476475,02:11
3,1.308329,0.804451,0.740851,02:11
4,1.183454,0.685591,0.794623,02:10


Labeling Smoothing

In [18]:
model = xresnet50(n_out = dls.c)
learn = Learner(dls, model, loss_func = LabelSmoothingCrossEntropy(), metrics = accuracy)
learn.fit_one_cycle(5,3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.807101,2.464983,0.408140,02:11
1,1.537389,1.391057,0.639283,02:19
2,1.309768,1.276487,0.687827,02:15
3,1.141487,1.082402,0.771471,02:12
4,1.026079,0.998856,0.810680,02:12
